In [1]:
!pip install PyPDF2
!pip install sentence_transformers
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 5.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.62.2
    Uninstalling grpcio-1.62.2:
      Successfully uninstalled grpcio-1.62.2
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

In [ ]:
from sentence_transformers import SentenceTransformer
def get_sentenceTF_embeddings(sentences):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  embeddings =[]
  for chunk in sentences:
    embeddings.append(model.encode(chunk))
  print(len(embeddings))
  return embeddings

def Embed_stenteceTF(sentence):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  return model.encode(sentence)

In [3]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans


def get_bert_embeddings(sentences):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize sentences
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token representation as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    return embeddings

def semantic_chunking_with_attention(text, n_clusters=5, max_chunk_size=512):
    # Split the input text into sentences based on '.'
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]

    # Get BERT embeddings for sentences
    embeddings = get_bert_embeddings(sentences)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=5).fit(embeddings)
    labels = kmeans.labels_

    # Group sentences by cluster
    clusters = {}
    for sentence, label in zip(sentences, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(sentence)

    # Create chunks from clusters with max_chunk_size constraint
    chunks = []
    for cluster in clusters.values():
        current_chunk = ""
        for sentence in cluster:
            if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:  # +1 for the period
                current_chunk += sentence + ". "
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "
        if current_chunk:
            chunks.append(current_chunk.strip())

    return chunks

In [4]:
from PyPDF2 import PdfReader
import re

def read_pdf_file(file_path):
    points = []
    # Open the PDF file
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        # Iterate through each page in the PDF
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                points.append(text.strip())
    return points

def replace_double_spaces(text):
    # Replace all occurrences of non-breaking spaces and zero-width spaces
    cleaned_text = re.sub(r'\xa0', ' ', text)
    cleaned_text1 = re.sub(r'\u200b', '', cleaned_text)
    return cleaned_text1

def process_pdf_files(file_paths, chunking_type, max_chunk_size, number_of_clusters=20):
    all_embeddings = []
    all_chunks = []

    for file_path in file_paths:
        print(f"Processing: {file_path}")
        Pneumonia = read_pdf_file(file_path)
        
        # Clean the text
        for i in range(len(Pneumonia)):
            Pneumonia[i] = replace_double_spaces(Pneumonia[i])
        
        Pneumonia_Str = "\n".join(Pneumonia)
        
        # Chunking
        if chunking_type == "Recursive":
            chunks = Recursive_Chunking(Pneumonia_Str, chunk_size=max_chunk_size)
        elif chunking_type == "Semantic":
            chunks = semantic_chunking_with_attention(Pneumonia_Str, n_clusters=number_of_clusters, max_chunk_size=max_chunk_size)
        
        # Get embeddings
        embedding = get_sentenceTF_embeddings(chunks)
        
        all_embeddings.extend(embedding)
        all_chunks.extend(chunks)
    
    return all_embeddings, all_chunks

In [ ]:
# Provide the list of PDF file paths
file_paths = [
    '/kaggle/input/prospectus/UG_Final_WHOLE_2024-Latest_10-7-24.pdf'
]

# Process all the files and obtain embeddings and chunks
embeddings, chunks = process_pdf_files(file_paths, chunking_type='Semantic', max_chunk_size=600, number_of_clusters=20)

# Print the lengths of the chunks and embeddings
print(len(chunks))
print(len(embeddings))

Processing: /kaggle/input/prospectus/UG_Final_WHOLE_2024-Latest_10-7-24.pdf


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
from qdrant_client.models import PointStruct
import uuid

# Create PointStruct objects
points = [
    PointStruct(
        id=str(uuid.uuid4()),  # Use a unique ID for each document
        vector=embedding,  # Ensure this is a list of floats
        payload={"text": doc}  # Optional metadata
    )
    for i, (doc, embedding) in enumerate(zip(chunks, embeddings))
]

In [12]:
from qdrant_client import QdrantClient

url = "https://2c9a0424-6bc6-477d-bef1-e8d25b9f8f00.us-west-1-0.aws.cloud.qdrant.io:6333"
api_key = "CIsWjhjyUIqr8LvBvnKE3vmnYbmxQzyRvhtGubULavikWQ8wa3QkVQ"

# Initialize Qdrant client
qdrant_client = QdrantClient(url=url, api_key=api_key)

In [13]:
from qdrant_client.models import CollectionParams, VectorParams

# Define the dimension of your vectors and distance metric
vector_dim = 384  # Dimension of your vectors
distance_metric = "Cosine"  # Distance metric (use "Cosine" as string)

# Define vector configuration
vector_params = VectorParams(size=vector_dim, distance=distance_metric)

# Create the collection
qdrant_client.create_collection(
    collection_name="ir-assignment-06",
    vectors_config=vector_params  # Correct argument name
)

True

In [14]:
# Insert points into Qdrant
qdrant_client.upsert(
    collection_name="ir-assignment-06",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
query = ["Department of Computer Science"]
query_embedding_response = get_sentenceTF_embeddings(query)
# print(query_embedding_response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1


In [16]:
# Assuming query_embedding_response is a list of lists or a NumPy array
query_embedding = query_embedding_response[0].tolist()  # Convert to list if necessary

In [17]:
# Perform similarity search
results = qdrant_client.search(
    collection_name="ir-assignment-06",
    query_vector=query_embedding,
)
# Print results
for result in results:
    print(result)

id='5d80ece6-a0a8-43bd-9fe1-2d6f13c153b8' version=0 score=0.5760531 payload={'text': 'Aaliya Farooq  \n 3 \nIntroduction  4 \nThe Department of Computer Science is one of the prominent and oldest centers of computer education in the country. Its histo ry dates back to 1968 when UET 5 \nLahore established a Computer Center under the supervision of Department of Mathematics. The center was respons ible for teaching of courses in 7 \nComputer Science and Numerical Analysis, which formed an integral part of the curricula  for aII disciplines of B. The 8 \ncenter also offered short term computer courses for private and pubic  sector organizations.'} vector=None shard_key=None order_value=None
id='086c42d5-7d2d-4977-92a0-4ca01f430c22' version=0 score=0.5690288 payload={'text': '5 \nThe Department also organizes seminars and workshops frequently in various areas of electrical, electronics, computer, and co ntrol engineering. Computer Science in 1978, that was the first graduate program of the